In [17]:
import json
import requests
import pandas as pd
from config import key_dict
from pprint import pprint
api_key = key_dict['oiko_weather']


#load hundred_city_df
hundred_city_df = pd.read_csv (r'Resources\100_countiesdata.csv')
#clean up hundred_city_df
hundred_city_df = hundred_city_df.drop(labels=["county_ascii"],axis=1)
hundred_city_df.rename(columns={"county_x":"county"},inplace=True)
#create weather_df
column_names=['county','state_name','county_fips','state_id','lat','lng','population']
weather_df = pd.DataFrame(columns = column_names)
weather_df, hundred_city_df

(Empty DataFrame
 Columns: [county, state_name, county_fips, state_id, lat, lng, population]
 Index: [],
        FIPS      county  county_fips state_id    state_name      lat  \
 0   31117.0   McPherson        31117       NE      Nebraska  41.5682   
 1   31105.0     Kimball        31105       NE      Nebraska  41.1978   
 2   27013.0  Blue Earth        27013       MN     Minnesota  44.0346   
 3   41015.0       Curry        41015       OR        Oregon  42.4576   
 4   41033.0   Josephine        41033       OR        Oregon  42.3655   
 ..      ...         ...          ...      ...           ...      ...   
 95   8089.0       Otero         8089       CO      Colorado  37.9026   
 96  42031.0     Clarion        42031       PA  Pennsylvania  41.1924   
 97  16041.0    Franklin        16041       ID         Idaho  42.1811   
 98  53031.0   Jefferson        53031       WA    Washington  47.7489   
 99  50023.0  Washington        50023       VT       Vermont  44.2734   
 
          lng  po

In [30]:
### THIS IS THE REAL DEAL API CALL
for index, row in hundred_city_df.iterrows():
    #print(row)
    lat = row['lat']
    lng = row['lng']
    county_fips = row['county_fips']
    county = row['county']
    state = row['state_name']
    state_id = row['state_id']
    population = row['population']
    ###---  The Request!!!
    r = requests.get('https://api.oikolab.com/weather',
                     params={'param': ['temperature','wind_speed','surface_solar_radiation',
                                       'total_cloud_cover', 'total_precipitation',
                                       'relative_humidity'],
                             'start': '2020-01-21',
                             'end': '2021-03-17',
                             'freq':"D",
                             'lat': lat,
                             'lon': lng,
                             'api-key': api_key})                 
    ## store json resonse per call in weather_df
    #print(index, row, r)
    weather_data = json.loads(r.json()['data'])
    #make a temp dataframe to pass the data
    temp_df = pd.DataFrame(
                  index=pd.to_datetime(weather_data['index'],unit='s'),
                  data=weather_data['data'],
                  columns=weather_data['columns'])
    temp_df.reset_index(inplace=True)
    temp_df = temp_df.rename(columns={'index':'date'})
    print(temp_df)
                
    #append the weather data to weather_df
    weather_df = weather_df.append(
                    other=temp_df,
                    ignore_index=True)
    #assign county, state, county_fips, state_id, and date to weather dataframe
    weather_df.loc[index, 'county'] = county
    weather_df.loc[index, 'state_name'] = state
    weather_df.loc[index, 'county_fips'] = county_fips
    weather_df.loc[index, 'state_id'] = state_id
    weather_df.loc[index, 'population'] = population
    weather_df.loc[index, 'lat'] = lat
    weather_df.loc[index, 'lng'] = lng
    break
weather_df

          date  temperature (degC)  wind_speed (m/s)  \
0   2020-01-21                2.46              2.38   
1   2020-01-22                1.88              7.77   
2   2020-01-23               -1.02              2.80   
3   2020-01-24                1.36              1.80   
4   2020-01-25                2.96              3.63   
..         ...                 ...               ...   
417 2021-03-13                2.72              9.64   
418 2021-03-14                1.68              2.38   
419 2021-03-15                2.00              2.02   
420 2021-03-16                0.88              5.49   
421 2021-03-17                3.07              7.30   

     surface_solar_radiation (W/m^2)  total_cloud_cover (0-1)  \
0                                101                     0.84   
1                                 98                     0.70   
2                                118                     0.44   
3                                126                     0.41   
4 

,county,state_name,county_fips,state_id,lat,lng,population,date,temperature (degC),wind_speed (m/s),surface_solar_radiation (W/m^2),total_cloud_cover (0-1),total_precipitation (mm of water equivalent),relative_humidity (0-1)
0,McPherson,Nebraska,31117,NE,41.5682,-101.061,395,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,Kimball,Nebraska,31105,NE,41.1978,-103.715,3633,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,Blue Earth,Minnesota,27013,MN,44.0346,-94.067,66795,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,Curry,Oregon,41015,OR,42.4576,-124.157,22650,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.46,2.38,101.0,0.84,0.0,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-13 00:00:00,2.72,9.64,35.0,1.00,0.0,0.96
844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-14 00:00:00,1.68,2.38,155.0,0.97,0.0,0.81
845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-15 00:00:00,2.00,2.02,145.0,0.94,0.0,0.80
846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-16 00:00:00,0.88,5.49,145.0,0.55,0.0,0.72


In [42]:
### Example Weather API Call
# for index, row in hundred_city_df.iterrows():
#     #print(row)
#     lat = row['lat']
#     lng = row['lng']
#     county_fips = row['county_fips']
#     county = row['county']
#     state = row['state']
#     state_id = row['state_id']
#     population = row['population']
#     #the request
#     r = requests.get('https://api.oikolab.com/weather',
#                      params={'param': ['temperature'
#                                         ],
#                              'start': '2020-01-21',
#                              'end': '2021-03-17',
#                              'freq':"D",
#                              'lat': lat,
#                              'lon': lng,
#                              'api-key': api_key})                 
    ## store json resonse per call in weather_df
#     print(r)
#     weather_data = json.loads(r.json()['data'])
#     weather_df.loc[index, 'temperature' ] = weather_data['data'][0]
#     weather_df.loc[index, 'urban_temperature'] = weather_data['data'][1]
#     weather_df.loc[index, 'skin_temperature'] = weather_data['data'][2]
#     weather_df.loc[index, 'wind_speed'] = weather_data['data'][3]
#     weather_df.loc[index, 'surface_solar_radiation'] = weather_data['data'][4]
#     weather_df.loc[index, 'total_cloud_cover'] = weather_data['data'][5]
#     weather_df.loc[index, 'total_precipitation'] = weather_data['data'][6]
#     weather_df.loc[index, 'relative_humidity'] = weather_data['data'][7]
#     weather_df.loc[index, 'county'] = county
#     weather_df.loc[index, 'state'] = state
#     weather_df.loc[index, 'county_fips'] = county_fips
#     weather_df.loc[index, 'state_id'] = state_id
    
#     break
# weather_df
#weather_df.to_csv('weather.csv', index=False)
#nested for loop
#for each call, loop over temperature and dates. 

<Response [200]>


,county,state,county_fips,state_id,lat,lng,population,temperature,urban_temperature,skin_temperature,wind_speed,surface_solar_radiation,total_cloud_cover,total_precipitation,relative_humidity
0,Chowan,North Carolina,37041,NC,NaN,NaN,NaN,[1.95],[4.73],[10.43],[14.05],[7.78],[6.86],[8.14],[5.26]


In [41]:
json.loads(r.json()['data'])

{'columns': ['temperature (degC)'],
 'index': [1579564800,
  1579651200,
  1579737600,
  1579824000,
  1579910400,
  1579996800,
  1580083200,
  1580169600,
  1580256000,
  1580342400,
  1580428800,
  1580515200,
  1580601600,
  1580688000,
  1580774400,
  1580860800,
  1580947200,
  1581033600,
  1581120000,
  1581206400,
  1581292800,
  1581379200,
  1581465600,
  1581552000,
  1581638400,
  1581724800,
  1581811200,
  1581897600,
  1581984000,
  1582070400,
  1582156800,
  1582243200,
  1582329600,
  1582416000,
  1582502400,
  1582588800,
  1582675200,
  1582761600,
  1582848000,
  1582934400,
  1583020800,
  1583107200,
  1583193600,
  1583280000,
  1583366400,
  1583452800,
  1583539200,
  1583625600,
  1583712000,
  1583798400,
  1583884800,
  1583971200,
  1584057600,
  1584144000,
  1584230400,
  1584316800,
  1584403200,
  1584489600,
  1584576000,
  1584662400,
  1584748800,
  1584835200,
  1584921600,
  1585008000,
  1585094400,
  1585180800,
  1585267200,
  1585353600,
  1

<Response [200]>


'{"title": "Location not specified", "description": "Please check the location parameters", "link": {"text": "Documentation related to this error", "href": "https://oikolab.com/api-details#api=weather&operation=weather-data", "rel": "help"}}'